In [40]:
import os
import tensorflow as tf
from tensorflow.keras import Input, layers, Model
import time
import pandas as pd

In [41]:
class OCRDataLoader():
    def __init__(self,train_path,
                 batch_size=64, repeat=1):
        """ Loads the tf.data.Dataset 
        Parameters:
            -annotation_path :  Path of image folder to be loaded in dataset
            -labels_path :  path of labels.txt file
            -image_width : image width 
            -table_path  : path of the table.txt file
            -batch_size  : Batch size 
            -repeat      : The number of times we want data to repeat. e.g if repeat 1 data is iterted once.
            
        """
        images_path=os.path.join(train_path,"images")
        labels_path=os.path.join(train_path,"labels")
        img_paths, label_paths = read_img_paths_and_labels(
            images_path, 
            labels_path)
        self.batch_size = batch_size
        self.size = len(img_paths)


        ds = tf.data.Dataset.from_tensor_slices((img_paths, label_paths))
        ds = ds.map(self._decode_and_resize)
        ds = ds.apply(tf.data.experimental.ignore_errors())
        ds = ds.batch(batch_size).repeat(repeat)
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
        self.dataset = ds

    def _decode_and_resize(self, image_filename, label_filename):
        """ Reads and resize image (64,image_width)
            Parameters:
                -filename : Name of file.
                -label   : label of that image.
                
            returns image tensor and label.
        """
        image = tf.io.read_file(image_filename)
        image = tf.io.decode_png(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        print("image:\t")
        print(image.shape)
        image = tf.image.resize(image, (2048,1024))
        label = tf.io.read_file(label_filename)
        label = tf.io.decode_png(label, channels=3)
        print(label.shape)
        label = tf.image.resize(label, (2048,1024))
        label = tf.cast((label[:,:,0]>100), tf.int32)
#         label = tf.image.convert_image_dtype(label, tf.float32)
        label = tf.one_hot(label, 2) #change to n_classes
        return image, label

    def __call__(self):
        """Return tf.data.Dataset."""
        return self.dataset
    def __len__(self):
        return self.size

def read_img_paths_and_labels(images_path, labels_path):
    """return image filenames  and respective labels.
       Parameters:
           -data_path : Path of the folder of images.
           -labels_path : path of label.txt
    
    """
    img_path=os.listdir(images_path)
    img_path.sort()
    img_path=[str(os.path.join(images_path,i)) for i in img_path]
    label_path=os.listdir(labels_path)
    label_path.sort()
    label_path=[str(os.path.join(labels_path,i)) for i in label_path]
    print(img_path,label_path)
    
    return img_path, label_path

In [42]:
x=OCRDataLoader("/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/",batch_size=2)

['/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/images/0.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/images/1.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/images/2.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/images/3.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/images/4.png'] ['/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/0.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/1.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/2.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/3.png', '/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/4.png']
image:	
(None, None, 3)
(None, None, 3)


In [20]:
def jitter(x,y):
        x=tf.image.adjust_brightness(x,150)
        x=tf.image.adjust_contrast(x,100)
        x=tf.image.adjust_hue(x,100)
        x=tf.image.adjust_saturation(x,200)
        x=tf.image.rot90(x)

        return (x,y)

In [45]:
class augmentation():
    def __init__(self,ds,prob=0.02):
        self.dataset=ds
        self.prob=prob
    def jitter(self,x,y):
        if np.random.choice(np.linspace(start=0.0,stop=1.0,num=100))<=self.prob:
            x=tf.image.adjust_brightness(x,np.random.choice(np.linspace(start=0.0,stop=1.0,num=100)))
            x=tf.image.adjust_contrast(x,np.random.choice(np.linspace(start=0.0,stop=1.0,num=100)))
            x=tf.image.adjust_hue(x,np.random.choice(np.linspace(start=0.0,stop=1.0,num=100)))
            x=tf.image.adjust_saturation(x,np.random.choice(np.linspace(start=0.0,stop=1.0,num=100)))
        return (x,y)
    def __call__(self):
        return self.dataset().map(self.jitter)

In [46]:
import numpy as np

In [47]:
d=augmentation(x)

In [48]:
for i ,j in d():
    print(tf.shape(i),tf.shape(j))
    break

tf.Tensor([   2 2048 1024    3], shape=(4,), dtype=int32) tf.Tensor([   2 2048 1024    2], shape=(4,), dtype=int32)


In [49]:
for i,j in x():
    break

In [50]:
i

<tf.Tensor: shape=(2, 2048, 1024, 3), dtype=float32, numpy=
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],




In [5]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(100, 100))
row=1
itter=1
for i,j in a:
    fig.add_subplot(10,2,itter)
    plt.imshow(i[0])
    fig.add_subplot(10,2,itter+1)
    plt.imshow(j[0][:,:,1])
    
    itter+=2
plt.show()
    
    
    

NameError: name 'a' is not defined

<Figure size 7200x7200 with 0 Axes>

In [13]:
def bottleneck(input_tensor,intial,filter_,stride=(1,1),connect=True,is_first=False):
    x=layers.Conv2D(filters=filter_,kernel_size=1,strides=(1,1),activation='relu')(input_tensor)
    x=layers.Conv2D(filters=filter_,kernel_size=3,strides=stride,padding="same",activation='relu')(x)
    x=layers.Conv2D(filters=intial,kernel_size=1,strides=(1,1))(x)
    
#     print(x.shape)
#     print(input_tensor.shape)
    if connect:
        if is_first:
#             print("here")
            input_tensor=layers.Conv2D(filters=intial,kernel_size=1,strides=(1,1))(input_tensor)
#             print("inside", input_tensor.shape)
        print(x.shape,input_tensor.shape)
        return x+input_tensor
    else:
        print("not Conected")
        print(x.shape)
    return x

            

In [14]:
def upsampling(input_layer,concat_layer,filters):
    x=layers.Conv2DTranspose(input_layer.shape[-1],kernel_size=1, strides=(2,2))(input_layer)
    #print(concat_layer.shape, x.shape)
    x=tf.concat([concat_layer,x],3)
    #print(x.shape)
    x=layers.Conv2D(filters,kernel_size=3,padding='same',activation='relu')(x)
    return x
    

In [15]:
def dh(num_clases):
    input_tensor=Input(shape=(2048,1024,3))
    x1=layers.Conv2D(filters=64,kernel_size=7,strides=(2,2), padding="same")(input_tensor)
    x2=layers.MaxPool2D(strides=(2,2))(x1)
    
    
    x3=bottleneck(x2,256,64,connect=True,is_first=True)
    x4=bottleneck(x3,256,64)
    x5=bottleneck(x4,256,64,(2,2),False)
    
    
    x6=bottleneck(x5,512,128,is_first=True)
    x7=bottleneck(x6,512,128)
    x8=bottleneck(x7,512,128)
    x9=bottleneck(x8,512,128,(2,2),False)
    
    x10=bottleneck(x9,1024,256,is_first=True)
    x11=bottleneck(x10,1024,256)
    x12=bottleneck(x11,1024,256)
    x13=bottleneck(x12,1024,256)
    x14=bottleneck(x13,1024,256)
    x15=bottleneck(x14,1024,256,(2,2),False)
    
    
    x16=bottleneck(x15,2048,512,is_first=True)
    x17=bottleneck(x16,2048,512)
    x18=bottleneck(x17,2048,512)
    
    
    y1=layers.Conv2D(512,kernel_size=1,strides=(1,1),padding="same")(x14)
    
    z1=layers.Conv2D(512,kernel_size=1,strides=(1,1),padding="same")(x18)
    
    z2=upsampling(z1,y1,512)
    z3=upsampling(z2,x8,256)
    z4=upsampling(z3,x4,128)
    z5=upsampling(z4,x1,64)
    z6=upsampling(z5,input_tensor,32)
    
    z7=layers.Conv2D(filters=num_clases,kernel_size=1,padding='same')(z6)
    
    
    
    
    

    
    
    
    return Model(inputs=input_tensor,outputs=z7,name="dh")
    

In [16]:
from tensorflow.keras import Input,layers,Model
import tensorflow as tf
a=dh(2)


(None, 512, 256, 256) (None, 512, 256, 256)
(None, 512, 256, 256) (None, 512, 256, 256)
not Conected
(None, 256, 128, 256)
(None, 256, 128, 512) (None, 256, 128, 512)
(None, 256, 128, 512) (None, 256, 128, 512)
(None, 256, 128, 512) (None, 256, 128, 512)
not Conected
(None, 128, 64, 512)
(None, 128, 64, 1024) (None, 128, 64, 1024)
(None, 128, 64, 1024) (None, 128, 64, 1024)
(None, 128, 64, 1024) (None, 128, 64, 1024)
(None, 128, 64, 1024) (None, 128, 64, 1024)
(None, 128, 64, 1024) (None, 128, 64, 1024)
not Conected
(None, 64, 32, 1024)
(None, 64, 32, 2048) (None, 64, 32, 2048)
(None, 64, 32, 2048) (None, 64, 32, 2048)
(None, 64, 32, 2048) (None, 64, 32, 2048)


In [ ]:
train_dl=OCRDataLoader("/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/",batch_size=1)

In [46]:
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        print(logits.shape)
        logit_length = tf.fill([tf.shape(logits)[0]], tf.shape(logits)[1])
        print(logit_length)
        loss = tf.nn.softmax_cross_entropy_with_logits(y,logits)
        loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss ,logits,logit_length

def train(model, optimizer, dl, epoch,log_freq=10):
    avg_loss = tf.keras.metrics.Mean(name="loss", dtype=tf.float32)
    steps = round(len(dl) / batch_size) + 1
    
    for i, (x, y) in enumerate(dl()):
        loss,logits,logit_length = train_one_step(model, optimizer, x, y)
        avg_loss.update_state(loss)
        if tf.equal(optimizer.iterations % log_freq, 0):
            print("{:.0%} Training... loss: {:.6f}  ".format(
                (i + 1) / steps, avg_loss.result()), end='')
            tf.summary.scalar("loss", avg_loss.result(), 
                              step=optimizer.iterations)
            avg_loss.reset_states()
            if i < steps - 1:
                print(end='\r')
        if epoch%10==0 and i%50==0:
            
            decoded, neg_sum_logits = tf.nn.ctc_greedy_decoder(
        inputs=tf.transpose(logits, perm=[1, 0, 2]),
        sequence_length=logit_length,
        merge_repeated=True)
            #print(tf.sparse.to_dense(decoded[0], default_value=0).numpy()
      
    return df

def val_one_step(model, x, y):
    logits = model(x, training=False)
    logit_length = tf.fill([tf.shape(logits)[0]], tf.shape(logits)[1])
    loss = tf.nn.ctc_loss(
        labels=y,
        logits=logits,
        label_length=None,
        logit_length=logit_length,
        logits_time_major=False,
        blank_index=-1)
    loss = tf.reduce_mean(loss)
    decoded, neg_sum_logits = tf.nn.ctc_greedy_decoder(
        inputs=tf.transpose(logits, perm=[1, 0, 2]),
        sequence_length=logit_length,
        merge_repeated=True)
    return loss, decoded

def val(model, dl, step):
    avg_loss = tf.keras.metrics.Mean(name="loss", dtype=tf.float32)
    steps = round(len(dl) / batch_size) + 1
    total_cnt = 0
    for i, (x, y) in enumerate(dl()):
        loss, decoded = val_one_step(model, x, y)
        avg_loss.update_state(loss)
        cnt = map_and_count(decoded[0], y, dl.inv_table, dl.blank_index)
        total_cnt += cnt
        t.append(tf.sparse.to_dense(y,default_value=0).numpy())
        p.append(tf.sparse.to_dense(decoded[0],default_value=0).numpy())
        t_.append(map_to_chars(y,val_dl.inv_table))
        p_.append(map_to_chars(decoded[0],val_dl.inv_table))
    
    
    accuracy = total_cnt / len(dl)
    print("Total: {}, Accuracy: {:.2%}".format(total_cnt, accuracy))
    tf.summary.scalar("loss", avg_loss.result(), step=step)
    tf.summary.scalar("accuracy", accuracy, step=step)
    avg_loss.reset_states()
    return df


In [25]:
import tensorflow as tf

In [27]:
import numpy as np

In [29]:
y=np.array([1,1,0,1,1]).astype("float")

In [31]:
logits=np.array([1,0,1,0,1]).astype('float')

In [39]:
loss = tf.nn.softmax_cross_entropy_with_logits(y,logits,weights=(1,700))

TypeError: softmax_cross_entropy_with_logits_v2() got an unexpected keyword argument 'weights'

In [34]:
c=tf.nn.softmax_cross_entropy_with_logits

In [36]:
l=c(y,logits)

In [38]:
tf.reduce_mean()

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
tf.summary.scalar("loss", avg_loss.result(), 
               `               step=optimizer.iterations)

In [ ]:
val_dl=None
batch_size=1

In [3]:
import time
type(time.asctime())


str

In [49]:
if __name__ == "__main__":
    localtime = time.asctime()
    print("Start at {}".format(localtime))

    model = dh(2)
    model.summary()

    """ lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=1000,
        decay_rate=0.96,`
        staircase=True)"""
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
    
    manager = tf.train.CheckpointManager(
        checkpoint, 
        directory="tf_ckpts/{}".format(localtime), 
        max_to_keep=5)
    checkpoint.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch")

    train_summary_writer = tf.summary.create_file_writer(
        f"logs/{localtime}/train")
    val_summary_writer = tf.summary.create_file_writer(
        f"logs/{localtime}/val")
    """ epochs=1
    for epoch in range(1, epochs + 1):
        print("Epoch {}:".format(epoch))
        with train_summary_writer.as_default():
            df=train(model, optimizer, train_dl,epoch)
        if True:
            checkpoint_path = manager.save(optimizer.iterations)
            print("Model saved to {}".format(checkpoint_path))
            if val_dl is not None:
                with val_summary_writer.as_default():
                    val(model, val_dl, optimizer.iterations)"""

Start at Thu Oct  1 15:11:21 2020
(None, 128, 64, 512) (None, 128, 64, 512)
(None, 128, 64, 1024)
(None, 256, 128, 512) (None, 256, 128, 512)
(None, 256, 128, 1024)
(None, 512, 256, 256) (None, 512, 256, 256)
(None, 512, 256, 512)
(None, 1024, 512, 64) (None, 1024, 512, 128)
(None, 1024, 512, 192)
(None, 2048, 1024, 3) (None, 2048, 1024, 64)
(None, 2048, 1024, 67)
Model: "dh"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 1024,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 512, 64 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D

In [ ]:
list(train_dl().as_numpy_iterator())[0][0].shape

In [ ]:
import tensorflow as tf


In [ ]:
import tensorflow as tf


In [ ]:
import numpy as np

In [ ]:
a=tf.keras.losses.MSE(tf.constant([1,2,3,4]),tf.constant([1,2,4,1]))

In [ ]:
import tensorflow as tf

In [ ]:
np.array([[0.2,0.8,0.0, 0.0],[0.1,0.2,0.7, 0.0]]).T

In [ ]:
tf.nn.softmax_cross_entropy_with_logits(y_hat, y)

In [ ]:
tf.nn.softmax_cross_entropy_with_logits([[0,1], [1,0]],np.array([[0.9,0.1],[0.1, 0.9]]).T)

In [ ]:
tf.nn.softmax_cross_entropy_with_logits_v2_helper

In [ ]:
y=np.array([0.01,1,0.1])
y_hat = y.copy()

In [ ]:
y*np.log(y_hat) + (1-y)*np.log(1-y_hat)

In [ ]:
yln(y^)+(1-y)ln(1-y^)

In [ ]:
tf.nn.softmax_cross_entropy_with_logits(np.array([[0.2,0.8,0.0, 0.0],[0.1,0.2,0.7, 0.0]]).T,[[1.,2]])

In [ ]:
! ls /mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/

In [ ]:
image_filename = "/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/labels/0.png"

In [ ]:
image = tf.io.read_file(image_filename)

image = tf.io.decode_png(image, channels=3)


In [ ]:
image[image.shape[0]//2:image.shape[0]//2+100,500:550,2]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image.numpy()

In [ ]:
plt.imshow(image.numpy()[:,:,0], "gray")

In [ ]:
np.unique(image.numpy())

In [ ]:
image[:,:,0]

In [ ]:
labels =np.array([[0,1,0]])

In [ ]:
logits = np.array([[[.9,.1], [0., 1], [0.9 ,0.1]]])

In [ ]:
labels

In [ ]:
logits

In [ ]:
tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(labels,2), logits=logits)

In [ ]:
tf.one_hot(label,2)

In [ ]:
labels

In [ ]:
label = tf.cast((image[:,:,0]>100), "int32")

In [ ]:
logits = np.random.rand(3,3,2)

In [ ]:
labels = np.array([[0,0,1],
         [1,0,1],
         [1,1,0]])

In [ ]:
tf.one_hot(label,2)

In [ ]:
tf.keras.metrics.Mean()

In [ ]:
import tensorflow as tf
a=tf.constant([1])


In [ ]:
str(a.numpy())

In [ ]:
import os
import tensorflow as tf

class DataLoader():
    def __init__(self,path,resize=(2048,1024),n_classes=2,batch_size=64,repeat=1):
        """ Loads the tf.data.Dataset 
        Parameters:
            -path        : Path of the Folder containing , 
                            images and labels folder
            -resize      : new size of images in a batch.
            -n_classes   : number of classes in labels.
            -batch_size  : Batch size 
            -repeat      : The number of times we want data to repeat. e.g if repeat 1 data is iterted once.
            
        """

        images_path=os.path.join(path,"images")
        labels_path=os.path.join(path,"labels")

        img_paths, label_paths = read_img_paths_and_labels(
            images_path, 
            labels_path)
        self.resize = resize
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.size = len(img_paths)


        ds = tf.data.Dataset.from_tensor_slices((img_paths, label_paths))
        ds = ds.map(self._decode_and_resize)
        ds = ds.apply(tf.data.experimental.ignore_errors())
        ds = ds.batch(batch_size).repeat(repeat)
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
        self.dataset = ds

    def _decode_and_resize(self, image_filename, label_filename):
        """ Reads and resize image (64,image_width)
            Parameters:
                -image_filename : Name of image file.
                -label_filename : Name of label file.
                
            returns image tensor and label tensor.
        """
        image = tf.io.read_file(image_filename)
        image = tf.io.decode_png(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        #image = tf.image.resize(image,self.resize)
        label = tf.io.read_file(label_filename)
        label = tf.io.decode_png(label, channels=3)
        #label = tf.image.resize(label,self.resize)
        label = tf.cast((label[:,:,0]>100), tf.int32)
        label = tf.one_hot(label, self.n_classes)
        return image, label

    def __call__(self):
        """Return tf.data.Dataset."""
        return self.dataset
    def __len__(self):
        return self.size

def read_img_paths_and_labels(images_path, labels_path):
    """return image filenames  and respective label filenames.
       Parameters:
           -images_path : Path of the folder of images.
           -labels_path : path of the folder of images.
    
    """
    img_path=os.listdir(images_path)
    img_path.sort()
    img_path=[str(os.path.join(images_path,i)) for i in img_path]
    label_path=os.listdir(labels_path)
    label_path.sort()
    label_path=[str(os.path.join(labels_path,i)) for i in label_path]
    
    return img_path, label_path

train_dl=DataLoader("/mnt/sda3/code/vscode/project/DataGenv5/DataGen/eval1/",batch_size=1)

for i in train_dl():
    continue
    

In [ ]:
!pip install tensorflow_addons

In [ ]:
a=tf.image.random_hue(i[1][0],0.05)

In [ ]:
import cv2
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(a)
plt.show()

In [ ]:
a[:,:,0]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.array(i[1][0])[:,:,1])
plt.show()

In [ ]:
i[1][0]

In [20]:
localtime = localtime = time.asctime()
print("Start at {}".format(localtime))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
    
manager = tf.train.CheckpointManager(
        checkpoint, 
        directory="tf_ckpts/{}".format(localtime), 
        max_to_keep=5)

Start at Thu Oct  1 11:42:05 2020


In [5]:
import tensorflow as tf

In [19]:
model=dh(2)

(None, 128, 64, 512) (None, 128, 64, 512)
(None, 128, 64, 1024)
(None, 256, 128, 512) (None, 256, 128, 512)
(None, 256, 128, 1024)
(None, 512, 256, 256) (None, 512, 256, 256)
(None, 512, 256, 512)
(None, 1024, 512, 64) (None, 1024, 512, 128)
(None, 1024, 512, 192)
(None, 2048, 1024, 3) (None, 2048, 1024, 64)
(None, 2048, 1024, 67)


In [22]:
print(manager.latest_checkpoint

In [24]:
if checkpoint.restore(manager.latest_checkpoint):
    print("yes")

yes
